# Deploying Your Model
In this exercise, we'll expose new images to our model and see how it does

## Loading the Model
Let's load the saved model that we trained. Modify the code below to point to the model you select.  

In [ ]:
from tensorflow import keras
import argparse
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import horovod.keras as hvd
from keras import backend as K
import keras.optimizers

In [ ]:
# note that you will need to update this model name 
model = keras.models.load_model('optimized-model/b2_e30/model_b2_e30_augtrue_fttrue.h5')

If you'd like to, you can see the summary of the model.

In [ ]:
model.summary()

### Showing the Images

When we use our model to make predictions on new images, it will be useful to show the image as well. We can use the matplotlib library to do this.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

In [ ]:
#uncomment the lines below to see a few of the testing set images
#Can change to pokemon images

data_dir = '/home/woodm/CSC2611/pokemon-image-classifier/data/'

# show_image(data_dir + 'Abra/00000000.png')

### Scaling the Images

The images in our dataset were 224x224 pixels. We need to make sure to pass the same size images into our method for prediction. There are a few ways to edit images with Python, but Keras has a built-in utility that works well. 

In [ ]:
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

def load_and_scale_image(image_path):
    image = image_utils.load_img(image_path, target_size=(224,224))
    return image

In [ ]:
image = load_and_scale_image(data_dir + 'Abra/00000000.png')
# image = load_and_scale_image('../data/Alakazam/.png')
# image = load_and_scale_image('../data/.png')
# image = load_and_scale_image('../data/.png')
# image = load_and_scale_image('../data/.png')
# image = load_and_scale_image('../data/.png')
plt.axis('off')
plt.imshow(image)

### Preparing the Image for Prediction

Now that we have the right size image, we're close to being ready to pass it into our model for prediction. First we need to reshape our image to match the shape of the dataset the model was trained on. Before we can reshape, we need to convert our image into a more rudimentary format. We'll do this with a keras utility called image_to_array.

In [ ]:
image = image_utils.img_to_array(image)

In [ ]:
print(f"Original image shape: {image.shape}")


Now we can reshape our image and scale it (preprocess) to get it ready for prediction.

In [ ]:
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)

## Making Predictions

Okay, now we're ready to predict! This is done by passing our pre-processed image into the model's predict method. 

In [ ]:
prediction = model.predict(image)
print(prediction)

### Understanding the Prediction

The predictions are in the format of a length 6 array. Each element of the array is a probability between 0 and 1, representing the confidence for each category. Let's make it a little more readable. We can start by finding which element of the array represents the highest probability. Fill in the following cell using numpy to find the largest value.  

In [ ]:
np.argmax(prediction)

In [ ]:
import os

# Specify the path to the 'data' folder in the parent directory
data_path = "/home/woodm/CSC2611/pokemon-image-classifier/data"

# Check if the directory exists
if os.path.exists(data_path) and os.path.isdir(data_path):
    # Get all folder names in the specified path
    folders = [folder for folder in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, folder))]
    
    # Create a dictionary with incremental keys starting from 1
    dictionary = {index: folder for index, folder in enumerate(sorted(folders))}
    
    # Print the resulting dictionary
    print(dictionary)
    
else:
    print(f"Directory {data_path} does not exist or is not accessible.")

We can now pass in our prediction index to find the corresponding letter.

In [ ]:
dictionary[np.argmax(prediction)]

In the next cell, we demonstrate how to find all the files in a directory that match a given pattern (png images).  For each of the 6 classes (e.g., apple, rotten apple, etc.) in our testing data set, alter the code below to calculate the accuracy of your model on that class. Report the accuracy per class along with the total number of images in the testing set for that class.

In [ ]:
data_dir = '/home/woodm/CSC2611/pokemon-image-classifier/data'
valid_extensions = ('.png', '.jpg', '.jpeg')

# Initialize lists to store accuracy and class names
accuracies = []
class_names = []

# Iterate through all subdirectories in data_dir
for subdir in os.listdir(data_dir):
    current_dir = os.path.join(data_dir, subdir)

    # Ensure the path is a directory
    if not os.path.isdir(current_dir):
        continue

    # Use the directory name as the label
    label = os.path.basename(current_dir)
    class_names.append(label)

    total = 0
    correct = 0

    # Get all valid image files in the directory
    all_files = [f for f in os.listdir(current_dir) if f.lower().endswith(valid_extensions)]

    for file in all_files:
        total += 1
        file_path = os.path.join(current_dir, file)

        # Preprocess the image
        image = load_and_scale_image(file_path)
        image = image_utils.img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)

        # Make prediction
        prediction = model.predict(image)
        guess = dictionary[np.argmax(prediction)]
        print(f"File: {file}, Guess: {guess} Correct: {subdir}")

        if guess == label:
            correct += 1

    # Calculate accuracy for the current class
    accuracy = correct / total if total > 0 else 0
    accuracies.append(accuracy)
    print(f"Accuracy for {label}: {accuracy:.2f}")

# Print out all accuracies at the end
print("\nOverall Results:")
for class_name, accuracy in zip(class_names, accuracies):
    print(f"Class: {class_name}, Accuracy: {accuracy:.2f}")